In [22]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import logging

logging.getLogger('jieba').setLevel(logging.WARN)
logging.getLogger('fgclassifier').setLevel(logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import config
from sklearn.feature_extraction.text import TfidfVectorizer
from fgclassifier.baseline import Indie
from fgclassifier import classifiers

vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0.01, max_df=0.95, norm='l2')
model = Indie(vectorizer=vectorizer)
df = model.read('data/english.csv', seg_words=False, sample_n=None)

X_train, Y_train = model.split_xy(df[:8000])
X_validate, Y_validate = model.split_xy(df[8000:])

2018-11-11 22:13:02,514 [INFO] Reading data/english.csv..
2018-11-11 22:13:02,712 [INFO] Transform features...
2018-11-11 22:13:02,713 [INFO] Fitting feature vectorizer...
2018-11-11 22:13:15,576 [INFO] Fitted training features, vocabulary: 5022


In [ ]:
from collections import Counter

print('\n'.join(['%s \t %s' % (k, v) for k, v in
                 Counter(model.vectorizer.vocabulary_).most_common()[:100]]))

In [ ]:
all_avg_scores, all_scores = {}, {}
for cls in ['LinearDiscriminantAnalysis']:
    if cls.startswith('_'):
        continue
    Classifier = getattr(classifiers, cls) 
    model = Indie(classifier=Classifier,
                  vectorizer=model.vectorizer,
                  reducer=model.reducer)
    model.train(X_train, Y_train)
    avg_score, scores = model.validate(X_validate, Y_validate)
    all_avg_scores[cls] = avg_score
    all_scores[cls] = scores